In [1]:
import json
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import warnings
import collections

In [2]:
train = pd.read_csv('data/dota2_skill_train.csv', index_col='id')
test = pd.read_csv('data/dota2_skill_test.csv', index_col='id')

In [3]:
def extract_features_csv(match):
    row = []
    for i in record.keys():
        if type(record[i]) in [str, int, float]:
            row.append((i, record[i]))
    return collections.OrderedDict(row)

In [4]:
df_new_features = []
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        features = extract_features_csv(record)
        df_new_features.append(features)
        
df = pd.DataFrame.from_records(df_new_features[0:100]).set_index('id')

5450it [00:04, 1339.67it/s]

KeyboardInterrupt: 

    damage_targets
    final_items
    ability_upgrades
    level_up_times
    item_purchase_log
    series
    radiant_heroes
    dire_heroes

# damage_targets

In [40]:
%%time
train_damage_targets = []

with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        damage_targets = record['damage_targets']
        damage_targets['id'] = record['id']
        
        train_damage_targets.append(damage_targets)
train_damage_targets = pd.DataFrame(train_damage_targets).set_index('id').fillna(0).astype('int')

CPU times: user 34.9 s, sys: 3.67 s, total: 38.6 s
Wall time: 38.4 s


In [41]:
%%time
test_damage_targets = []

with open('data/dota2_skill_test.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        
        damage_targets = record['damage_targets']
        damage_targets['id'] = record['id']
        
        test_damage_targets.append(damage_targets)
test_damage_targets = pd.DataFrame(test_damage_targets).set_index('id').fillna(0).astype('int')

CPU times: user 15.8 s, sys: 1.41 s, total: 17.2 s
Wall time: 18.3 s


# item_purchase_log, final_items

In [42]:
%%time
train_id_items = []
train_id = []
import json
it = 0
with open('data/dota2_skill_train.jsonlines') as fin:
    for line in tqdm_notebook(fin):
        record = json.loads(line)
        mid_items = []
        for i in record['item_purchase_log']:
            mid_items.append(i['item_id'])
        train_id_items.append(mid_items)
        train_id.append(record['id'])
        
train_items_log = pd.DataFrame(train_id_items, index=train_id)
train_items_qual_count = pd.DataFrame(index=train_id)

for train_id in tqdm_notebook(train_items_log.index):
    x = train_items_log[train_items_log.index == train_id].values
    x = x[~np.isnan(x)]
    train_items_qual_count.at[train_id, 'x_len'] = len(x)
    qual_item = [items[items.index == i]['qual'].values[0] for i in x]
    for q_i in ['component', 'rare', 'epic', 'consumable', 'artifact', 'common', 'secret_shop']:
        train_items_qual_count.at[train_id, 'qual_item_{}_count'.format(q_i)] = qual_item.count(q_i)

NameError: name 'items' is not defined